## OK... LET CHOICE 2 DIFFERENT LABELS FROM DATA

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


  
url='../input/classification-of-class-on-basis-of-text/root2ai - Data.csv'
df = pd.read_csv(url)

U=list(df['Target'])
A = list(set(df['Target']))

#Let's chose 2 very different labels, for example Bigdata and Cyber Security

V = list(df['Text'])

R = [[V[i],U[i]] for i in range(len(U))]
new_frame = [i for i in R if i[1]== 'Bigdata' or i[1]== 'Cyber Security']
new_frame_texts = [i[0] for i in new_frame]
new_frame_labels = [i[1] for i in new_frame]
new_frame_labels_asreal = []
for i in new_frame_labels:
    if i == 'Bigdata':
      new_frame_labels_asreal.append(1.0)
    elif i == 'Cyber Security':
      new_frame_labels_asreal.append(0.0)
# Okey, now we have new dataframe with labels Bigdata = 1.0 and Cyber Security = 0.0
df = pd.DataFrame(data = {'Text':new_frame_texts , 'Target':new_frame_labels_asreal})

#Let's watch how distributed labels
number_of_bd = 0
number_of_cs = 0
for i in list(df['Target']):
  if float(i) == 1.0:
    number_of_bd+=1
  elif float(i) == 0.0:
    number_of_cs +=1

text1 = ''
for i in new_frame_texts:
  text1+=i

df


# SOME PREPROCESSING WITH **NLTK**

In [20]:
from nltk import WordNetLemmatizer
import re
import string

U = list(df['Text'])
lemmatizer = nltk.WordNetLemmatizer()
V = list(df['Target'])

U_f = []
for j in U:
  R = nltk.word_tokenize(j)
  Y = [lemmatizer.lemmatize(i) for i in R]
  StrY = " ".join(Y)
  U_f.append(StrY)

stop_words = stopwords.words("english")

def cleaning(data):
    
    filtered = re.sub("@\S+", " ", data)  # remove mentions
    filtered = re.sub("https*\S+", " ", filtered) # remove url
    filtered = re.sub("#\S+", " ", filtered) # remove hashtags
    filtered = re.sub("\d", " ", filtered) # remove all numbers
    filtered = re.sub('[%s]+' % re.escape(string.punctuation), ' ', filtered)  # remove punctuation
    filtered = re.sub('\n', ' ', filtered) # remove new lines       
    filtered = re.sub('\s{2,}',' ', filtered) # remove extra spaces
    
    filtered = filtered.lower()
    
    filtered = ' '.join([word for word in filtered.split(' ') if word not in stop_words and len(word)>1 and word.isalpha() == True])
    
    return filtered

df = pd.DataFrame(data = {'Text':[cleaning(i) for i in U_f], 'Target':V})

df


# Scikit-learn methods for creating train and test dataset's

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(df['Text'], 
                                                    df['Target'], test_size=0.1 ,
                                                    random_state=42)


X_train, X_test, y_train, y_test = X_train, X_test, np.asarray(y_train).astype('float32'), np.asarray(y_test).astype('float32')



vectorizer=TfidfVectorizer()
vectorizer.fit(df['Text'])
X_train_vectorized = vectorizer.transform(X_train)
X_train_vectorized = X_train_vectorized.toarray()


X_train_vectorized.shape


## FINALLY... REWORKED MODEL FOR THIRD TASK

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers




X_train_vectorized = keras.preprocessing.sequence.pad_sequences(X_train_vectorized, maxlen =80)

model = keras.Sequential()
model.add(layers.Embedding(10000, 64)) 
model.add(layers.Bidirectional(layers.LSTM(32, return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(32, return_sequences=False)))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(
    optimizer='adam',
              loss="binary_crossentropy",
              metrics=["accuracy"],
)
model.summary()

#my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=5)]


history = model.fit(X_train_vectorized, y_train, batch_size=16, epochs=50, validation_split=0.2, callbacks = [tf.keras.callbacks.EarlyStopping(patience=5)])



In [25]:
import numpy as np

X_test_vectorized = vectorizer.transform(X_test.apply(lambda x: np.str_(x)))
X_test_vectorized = X_test_vectorized.toarray()
score = model.evaluate(X_test_vectorized, y_test, verbose=0) 

print('Test score:', score[0]) 
print('Test accuracy:', score[1])

In [26]:
history_dict = history.history
history_dict.keys()
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()